In [1]:
import torch
import pandas

from torch import nn
from torch.autograd import Variable

input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 1
learning_rate = 0.001
graph_stat = Variable(torch.ones(input_parameters, layer_size))


class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.linear0 = torch.nn.Linear(layer_size, layer_size)
        self.linear1 = torch.nn.Linear(layer_size, layer_size)
        self.linear2 = torch.nn.Linear(layer_size, classification_size)

    def forward(self, x):
        x = self.linear0(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x


model = NeuralNet()

result = model(graph_stat)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


def load_dataset(filename: str) -> tuple[list, list]:
    df = pandas.read_csv(filename)
    df.reset_index()
    return [torch.Tensor(v) for v in df.drop('label', axis=1).to_numpy()[:, 1:]], \
        [torch.Tensor([v]) for v in df['label'].to_numpy()]


dataset, dataset_labels = load_dataset('dataset.csv')

bound = int(len(dataset) * 0.8)
train = list(zip(dataset[:bound], dataset_labels[:bound]))
test = list(zip(dataset[bound:], dataset_labels[bound:]))
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
total_step = len(train)
for epoch in range(num_epochs):
    for i, (image, label) in enumerate(train):
        # Move tensors to the configured device
        image = image.to(device)
        label = label.to(device)

        # Forward pass
        outputs = model(image)
        loss = criterion(outputs, label)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/10], Step [100/160], Loss: -0.0000
Epoch [2/10], Step [100/160], Loss: -0.0000
Epoch [3/10], Step [100/160], Loss: -0.0000
Epoch [4/10], Step [100/160], Loss: -0.0000
Epoch [5/10], Step [100/160], Loss: -0.0000
Epoch [6/10], Step [100/160], Loss: -0.0000
Epoch [7/10], Step [100/160], Loss: -0.0000
Epoch [8/10], Step [100/160], Loss: -0.0000
Epoch [9/10], Step [100/160], Loss: -0.0000
Epoch [10/10], Step [100/160], Loss: -0.0000


In [4]:
from sampo.schemas.graph import WorkGraph


def metric_vertex_count(wg: WorkGraph) -> float:
    return wg.vertex_count

def metric_min_children(wg: WorkGraph) -> float:
    return min((len(node.children) for node in wg.nodes if node.children))

def metric_max_children(wg: WorkGraph) -> float:
    return max((len(node.children) for node in wg.nodes if node.children))

def metric_min_parents(wg: WorkGraph) -> float:
    return min((len(node.parents) for node in wg.nodes if node.parents))

def metric_max_parents(wg: WorkGraph) -> float:
    return max((len(node.parents) for node in wg.nodes if node.parents))

In [15]:
from copy import deepcopy
from sampo.scheduler.multi_agency.multi_agency import Agent, Manager
from sampo.scheduler.topological.base import TopologicalScheduler
from sampo.scheduler.heft.base import HEFTScheduler, HEFTBetweenScheduler
import time
from sampo.generator.base import SimpleSynthetic
from sampo.scheduler.multi_agency.block_graph import BlockGraph, BlockNode

ss = SimpleSynthetic(256)
schedulers = [HEFTScheduler(), HEFTBetweenScheduler(), TopologicalScheduler()]
contractors = [ss.contractor(10)]

print('   NeuralNet ------ Multi-Agency')
print('Result | Time ---- Result | Time')

with torch.no_grad():
    for _ in range(len(test)):
        wg = ss.work_graph(top_border=100)
        bg = BlockGraph([BlockNode(wg)])
        # encode graph
        encoding = torch.Tensor([metric_vertex_count(wg),
                                 metric_min_children(wg),
                                 metric_max_children(wg),
                                 metric_min_parents(wg),
                                 metric_max_parents(wg)])

        nn_start = time.time()
        # predict scheduler
        outputs = model(encoding)
        _, predicted = torch.max(outputs.data, 0)
        # use predicted scheduler to generate schedule
        nn_result = schedulers[predicted].schedule(wg, contractors).execution_time
        nn_time = time.time() - nn_start

        agents = [Agent(str(scheduler), scheduler, deepcopy(contractors)) for scheduler in schedulers]
        manager = Manager(agents)
        ma_start = time.time()
        # run MA scheduler selection
        block_schedule = list(manager.manage_blocks(bg).values())
        ma_result = max(v.end_time for v in block_schedule)
        ma_time = time.time() - ma_start

        if schedulers[predicted] != block_schedule[0].agent.scheduler:
            print('Prediction mismatch')

        print(f'{nn_result} | {nn_time} ---- {ma_result} | {ma_time} ------------- {int(ma_time / nn_time * 100)}% time win | {int((1 - ma_result / nn_result) * 100)}% result loss')


   NeuralNet ------ Multi-Agency
Result | Time ---- Result | Time
1522 | 0.14272284507751465 ---- 1523 | 1.0730080604553223 ------------- 751% time win | 0% result loss
1010 | 0.09198164939880371 ---- 1011 | 0.5129594802856445 ------------- 557% time win | 0% result loss
Prediction mismatch
1208 | 0.10194587707519531 ---- 1206 | 0.5308809280395508 ------------- 520% time win | 0% result loss
345 | 0.036889076232910156 ---- 346 | 0.16800856590270996 ------------- 455% time win | 0% result loss
Prediction mismatch
1045 | 0.09304475784301758 ---- 1017 | 0.40797924995422363 ------------- 438% time win | 2% result loss
1125 | 0.10598325729370117 ---- 1126 | 0.571998119354248 ------------- 539% time win | 0% result loss
1359 | 0.1515965461730957 ---- 1360 | 0.8186850547790527 ------------- 540% time win | 0% result loss
455 | 0.04587817192077637 ---- 456 | 0.1875016689300537 ------------- 408% time win | 0% result loss
1231 | 0.12865686416625977 ---- 1232 | 0.773496150970459 ------------- 60